# United States Congress Stock Transactions Retrieval

In [1]:
# Import packages
import csv, json, zipfile
import requests, PyPDF2
import os
from datetime import datetime

In [2]:
# Define global variables
current_year = str(datetime.today().year)
current_directory = os.getcwd()
doc_id = ''

In [3]:
# Get last name of represenative from user input
represenative_name = input("Represenative's last name: ")
represenative_name = represenative_name.lower() # Convert name to lowercase for directory creation

house_zip_url = "https://disclosures-clerk.house.gov/public_disc/financial-pdfs/" + current_year + "FD.ZIP"
house_pdf_base_url = "https://disclosures-clerk.house.gov/public_disc/ptr-pdfs/" + current_year + "/"

zip_file_request = requests.get(house_zip_url)
new_zipfile_name = represenative_name + current_year + ".zip"

In [4]:
# Download and unzip corresponding file
with open(new_zipfile_name, 'wb') as f:
    f.write(zip_file_request.content)

with zipfile.ZipFile(new_zipfile_name) as z:
    z.extractall('.')

In [5]:
with open('2021FD.txt') as f:
    for line in csv.reader(f, delimiter='\t'):

        if line[1].lower() == represenative_name:
            date = line[7]
            doc_id = line[8]

            pdf_doc_source = requests.get(f"{house_pdf_base_url}{doc_id}.pdf")

            # Create directory for represenative if it doesn't exist
            if not os.path.exists(current_directory + "/" + represenative_name):
                try:
                    os.makedirs(current_directory + "/" + represenative_name)
                except OSError as exc:  # Guard against race condition
                    if exc.errno != zipfile.error.EEXIST:
                        pass

            # Write newest file under represenative's name
            # Need ability to save to a directory under the represenatives name.
            with open(represenative_name + f"/{doc_id}.pdf", 'wb+') as pdf_file:
                pdf_file.write(pdf_doc_source.content)